In [3]:
%load_ext autoreload
%autoreload 2

In [8]:
from environment import TrafficModel, Car
from util.network import create_braess_network, create_double_braess_network, create_cars

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import networkx as nx
from tqdm import trange

In [89]:
number_of_steps = 1000
car_counts = {(0, 3): 30}

network = nx.DiGraph([(0, 1), (0, 2), (1, 3), (2, 3)])

nx.set_edge_attributes(
    network,
    {
        (0, 1): (0, 1),
        (0, 2): (1, 0),
        (1, 3): (1, 0),
        (2, 3): (0, 1),
    },
    "latency_params",
)

In [90]:
alpha = [(0, 3)]
routes = [nx.all_simple_edge_paths(network, s, t) for s, t in alpha]

n, m = network.number_of_nodes(), network.number_of_edges()
T = len(alpha)

Js = [np.array([[network.edges[edge]['latency_params'][0] if edge in route else 0 for edge in network.edges] for route in s_t_routes]) for s_t_routes in routes]

K = -nx.incidence_matrix(network, oriented=True).toarray()[:-1]

L = -nx.incidence_matrix(network, edgelist=[(1, 2)], oriented=True).toarray()[:-1]

# M1 = np.concatenate([np.ones((m - n + 2)), np.zeros((m - n + 2))])

In [99]:
Js

[array([[0, 0, 1, 0],
        [0, 1, 0, 0]])]

In [93]:
L

array([[-0.],
       [ 1.],
       [-1.]])

In [94]:
K

array([[ 1.,  1., -0., -0.],
       [-1., -0.,  1., -0.],
       [-0., -1., -0.,  1.]])

In [95]:
if T == 1:
    D = np.zeros((0, T + m))
elif T == 2:
    pass
    # D = np.vstack([np.hstack([np.ones((Js[0].shape[0], 1)), Js[0]]), np.hstack([np.zeros((Js[1].shape[0], 1)), Js[1]])])
else: # T > 2
    pass

D

array([], shape=(0, 5), dtype=float64)

In [96]:
D1 = np.vstack([np.hstack([np.ones((Js[0].shape[0], 1)), Js[0]]), D, np.hstack([np.zeros((K.shape[0], T)), K])])
D1

array([[ 1.,  0.,  0.,  1.,  0.],
       [ 1.,  0.,  1.,  0.,  0.],
       [ 0.,  1.,  1., -0., -0.],
       [ 0., -1., -0.,  1., -0.],
       [ 0., -0., -1., -0.,  1.]])

In [100]:
np.linalg.det(D1)

-2.0

In [97]:
D2 = np.vstack([np.hstack([np.zeros((Js[0].shape[0], 1)), Js[0]]), D, np.hstack([L, np.zeros((K.shape[0], T - 1)), K])])
D2

array([[ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.],
       [-0.,  1.,  1., -0., -0.],
       [ 1., -1., -0.,  1., -0.],
       [-1., -0., -1., -0.,  1.]])

In [101]:
np.linalg.det(D2)

1.0

In [98]:
np.linalg.det(D1) / np.linalg.det(D2)

-2.0